In [70]:
import pandas as pd
from funpymodeling import status

In [71]:
LINKS_PATH = "../datasets/raw_data/links.csv"
MOVIES_PATH = "../datasets/raw_data/movies.csv"
RATINGS_PATH = "../datasets/raw_data/ratings.csv"
TAGS_PATH = "../datasets/raw_data/tags.csv"

In [72]:
linksdf = pd.read_csv(LINKS_PATH, sep = ",") 
linksdf

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [73]:
moviesdf = pd.read_csv(MOVIES_PATH, sep = ",")
moviesdf

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [74]:
ratingsdf = pd.read_csv(RATINGS_PATH, sep = ",")
ratingsdf

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [75]:
ratingsdf.query("userId == 1")

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
227,1,3744,4.0,964980694
228,1,3793,5.0,964981855
229,1,3809,4.0,964981220
230,1,4006,4.0,964982903


In [76]:
tagsdf = pd.read_csv(TAGS_PATH, sep = ",")
tagsdf

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


Once we observed the columns in each dataset, we need to pay attention to 2 datasets, the `movies.csv` dataset and the `ratings.csv` dataset. This two datasets are going to be the ones we use for the recommendation model.

In [77]:
status(moviesdf)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,movieId,0,0.0,0,0.0,9742,int64
1,title,0,0.0,0,0.0,9737,object
2,genres,0,0.0,0,0.0,951,object


In [78]:
status(ratingsdf)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,userId,0,0.0,0,0.0,610,int64
1,movieId,0,0.0,0,0.0,9724,int64
2,rating,0,0.0,0,0.0,10,float64
3,timestamp,0,0.0,0,0.0,85043,int64


I am going to add a new column called ***"year"*** where we have the year of release for each movie in `moviesdf`.

Furthermore, I am going to modify the movie´s titles in order to transform values from this format: `"Matrix, The (1999)"`, into this other format `"The Matrix"`

In [79]:
import re

In [80]:
def modify_text(match):
    text="("
    t=match.group(1)
    t=t.split(", ")
    t.reverse()
    for x in t:
        text += x
        if x != t[len(t[:-1])]:
            text += " "
        else:
            text += ")"
    return text

In [81]:
def extract_year(df):
    examples = ["the"," the", "a", " a", "an", " an"]
    year_of_release = []
    title_corrected = []
    for title in df:
        if title.endswith(" "):
            title=title[:-1]
        title = title.split(" ")
        try:
            year= title[-1].strip("()")
            if len(year) != 4:
                title.append(year)
            year = int(year) if len(year) == 4 else "nan"
        except:
            year = "nan"
        title.remove(title[-1])
        movie_title=""
        for x in range(len(title)):
            movie_title += title[x] + " "

        if movie_title.endswith(" "):
            movie_title=movie_title[:-1]
        for x in examples:
            pattern=","+ x
            if movie_title.lower().endswith(pattern):
                axis=movie_title.lower().rfind(pattern)
                movie_title = x.strip(" ").capitalize() + " " + movie_title[:axis]
        movie_title = movie_title[:-1] if movie_title.endswith(" ") else movie_title
        title_corrected.append(movie_title)
        year_of_release.append(year)
        
    return title_corrected, year_of_release

In [82]:
moviesdf["title"], moviesdf["year"] = extract_year(moviesdf["title"])

In [83]:
from funpymodeling import freq_tbl
freq_tbl(moviesdf["title"])

,title,frequency,percentage,cumulative_perc
0,Hamlet,5,0.000513,0.000513
1,The Three Musketeers,4,0.000411,0.000924
2,Jane Eyre,4,0.000411,0.001334
3,"Misérables, Les",4,0.000411,0.001745
4,A Christmas Carol,4,0.000411,0.002156
...,...,...,...,...
9443,"10th Victim, The (La decima vittima)",1,0.000103,0.999589
9444,Suspiria,1,0.000103,0.999692
9445,"Fist of Fury (Chinese Connection, The) (Jing w...",1,0.000103,0.999795
9446,Christiane F. (a.k.a. We Children from Bahnhof...,1,0.000103,0.999897


In [84]:
moviesdf

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018


In [85]:
freq_tbl(moviesdf["title"]).query("frequency == 2")

,title,frequency,percentage,cumulative_perc
21,The Freshman,2,0.000205,0.007391
22,Footloose,2,0.000205,0.007596
23,Bernie,2,0.000205,0.007801
24,Clash of the Titans,2,0.000205,0.008007
25,Eros,2,0.000205,0.008212
...,...,...,...,...
261,Crossroads,2,0.000205,0.056662
262,The Browning Version,2,0.000205,0.056867
263,The Girl Next Door,2,0.000205,0.057072
264,Focus,2,0.000205,0.057278


There are no **NaNs** so we can proceed to merge both datasets into a bigger one.

In [86]:
ratingsdf = pd.merge(ratingsdf,moviesdf[["title","movieId"]], on="movieId")

In [87]:
ratingsdf

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story
1,5,1,4.0,847434962,Toy Story
2,7,1,4.5,1106635946,Toy Story
3,15,1,2.5,1510577970,Toy Story
4,17,1,4.5,1305696483,Toy Story
...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon
100832,610,160527,4.5,1479544998,Sympathy for the Underdog
100833,610,160836,3.0,1493844794,Hazard
100834,610,163937,3.5,1493848789,Blair Witch


In [88]:
ratingsdf.query("userId == 1")

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story
215,1,3,4.0,964981247,Grumpier Old Men
267,1,6,4.0,964982224,Heat
369,1,47,5.0,964983815,Seven (a.k.a. Se7en)
572,1,50,5.0,964982931,The Usual Suspects
...,...,...,...,...,...
16056,1,3744,4.0,964980694,Shaft
16075,1,3793,5.0,964981855,X-Men
16208,1,3809,4.0,964981220,What About Bob?
16243,1,4006,4.0,964982903,Transformers: The Movie


In [89]:
status(ratingsdf)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,userId,0,0.0,0,0.0,610,int64
1,movieId,0,0.0,0,0.0,9724,int64
2,rating,0,0.0,0,0.0,10,float64
3,timestamp,0,0.0,0,0.0,85043,int64
4,title,0,0.0,0,0.0,9433,object


In [90]:
ratingsdf.shape

(100836, 5)

I am going to replace the `movieId` column for the `imdbId` column because when doing the gradio app, I will need the `imdbId` values instead of the `movieId` values.

In [91]:
linksdf = linksdf.drop("tmdbId",axis=1)

In [92]:
data = pd.merge(ratingsdf,linksdf,on="movieId")

In [93]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,userId,0,0.0,0,0.0,610,int64
1,movieId,0,0.0,0,0.0,9724,int64
2,rating,0,0.0,0,0.0,10,float64
3,timestamp,0,0.0,0,0.0,85043,int64
4,title,0,0.0,0,0.0,9433,object
5,imdbId,0,0.0,0,0.0,9724,int64


Now I am going to drop the `movieId` column

In [94]:
data = data.drop("movieId",axis=1)
data

,userId,rating,timestamp,title,imdbId
0,1,4.0,964982703,Toy Story,114709
1,5,4.0,847434962,Toy Story,114709
2,7,4.5,1106635946,Toy Story,114709
3,15,2.5,1510577970,Toy Story,114709
4,17,4.5,1305696483,Toy Story,114709
...,...,...,...,...,...
100831,610,2.5,1479545749,Bloodmoon,118745
100832,610,4.5,1479544998,Sympathy for the Underdog,66806
100833,610,3.0,1493844794,Hazard,798722
100834,610,3.5,1493848789,Blair Witch,1540011


In [95]:
data.head()

,userId,rating,timestamp,title,imdbId
0,1,4.0,964982703,Toy Story,114709
1,5,4.0,847434962,Toy Story,114709
2,7,4.5,1106635946,Toy Story,114709
3,15,2.5,1510577970,Toy Story,114709
4,17,4.5,1305696483,Toy Story,114709


In [96]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,userId,0,0.0,0,0.0,610,int64
1,rating,0,0.0,0,0.0,10,float64
2,timestamp,0,0.0,0,0.0,85043,int64
3,title,0,0.0,0,0.0,9433,object
4,imdbId,0,0.0,0,0.0,9724,int64


Next, lets drop the `timestamp` column

In [97]:
data = data.drop("timestamp", axis = 1)

In [98]:
data

,userId,rating,title,imdbId
0,1,4.0,Toy Story,114709
1,5,4.0,Toy Story,114709
2,7,4.5,Toy Story,114709
3,15,2.5,Toy Story,114709
4,17,4.5,Toy Story,114709
...,...,...,...,...
100831,610,2.5,Bloodmoon,118745
100832,610,4.5,Sympathy for the Underdog,66806
100833,610,3.0,Hazard,798722
100834,610,3.5,Blair Witch,1540011


Now that we are finished working with `data`, is time to do some transformations in how the genres are displayed in `moviesdf`

In [99]:
status(data)    

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,userId,0,0.0,0,0.0,610,int64
1,rating,0,0.0,0,0.0,10,float64
2,title,0,0.0,0,0.0,9433,object
3,imdbId,0,0.0,0,0.0,9724,int64


Now I transform each string into a list with strings, where each string is a different genre

In [100]:
genres = []
for genre in moviesdf["genres"]:
    movieGenres = []
    m = genre.split("|")
    for x in m:
        movieGenres.append(x)
    genres.append(movieGenres)

In [101]:
moviesdf["genres"] = genres

In [102]:
moviesdf

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]",2017
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]",2017
9739,193585,Flint,[Drama],2017
9740,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]",2018


Before saving both dataframes, I am going to replace `movieId` column in `moviesdf` for `imdbId` column.

In [103]:
moviesdf = pd.merge(moviesdf,linksdf,on="movieId")
moviesdf = moviesdf.drop("movieId",axis=1)
moviesdf

,title,genres,year,imdbId
0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,114709
1,Jumanji,"[Adventure, Children, Fantasy]",1995,113497
2,Grumpier Old Men,"[Comedy, Romance]",1995,113228
3,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,114885
4,Father of the Bride Part II,[Comedy],1995,113041
...,...,...,...,...
9737,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]",2017,5476944
9738,No Game No Life: Zero,"[Animation, Comedy, Fantasy]",2017,5914996
9739,Flint,[Drama],2017,6397426
9740,Bungo Stray Dogs: Dead Apple,"[Action, Animation]",2018,8391976


Finally, I am going to save the adapted dataset

In [104]:
data.to_csv("../datasets/adapted_data/adaptedRatingsData.csv", index = False)
moviesdf.to_csv("../datasets/adapted_data/adaptedFeaturesData.csv", index = False)